In [1]:
import copy, datetime, json, os, pprint, time, uuid
import vertexai
from google.cloud import storage

from google.genai.types import Part, UserContent, ModelContent
from google.adk.agents.llm_agent import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import google_search

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION,
              staging_bucket=f'gs://{PROJECT_ID}')

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

In [45]:
instruction = f'''
あなたの役割は、特定のトピックに関する気軽な読み物記事を書くことです。
ユーザーは、記事の「トピック」と、その内容に関連する「調査レポート」を提供します。
調査レポートに記載の客観的事実に基づいて、信頼性のある読み物記事を書いてください。

**処理の手順**
ユーザーが「トピック」と「調査レポート」の両方の情報を提供した後に記事を生成してください。
情報が不足している場合は、ユーザーに提供をお願いしてください。

**出力条件**
- トピックに関してある程度の基礎知識がある読者を前提として、数分で気軽に読める内容にしてください。
- 比較的カジュアルで語りかける口調の文章にします。
- 思考過程は出力せずに、最終結果だけを出力します。
- 次のような構成にします。各セクションのタイトルは、内容に合わせてアレンジしてください。
  タイトルには絵文字を入れてポップな見栄えにします。

1. 導入部分：なぜいまこのトピックを取り上げるのか、読者にこれを読む動機づけを与えます。
2. 概要：トピックの全体像をまとめて簡単に説明します。
3. 最新情報：特に注目したい新しい情報を取り上げます。
4. 実践：トピックに関して、読者自身がやってみるとよさそうな事を紹介します。
5. まとめ
'''

article_writer = LlmAgent(
    name='inspire_agent',
    model='gemini-2.5-flash-preview-05-20',
#    model='gemini-2.0-flash',
    description='特定のトピックに関する読み物記事を書くエージェント',
    instruction=instruction,
)

In [46]:
class LocalApp:
    def __init__(self, agent, app_name, user_id):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = self._runner.session_service.create_session(
            app_name=self._app_name,
            user_id=self._user_id,
            state={},
            session_id=uuid.uuid4().hex,
        )
        
    async def stream(self, query):
        content = UserContent(parts=[Part.from_text(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if DEBUG:
                print(f'====\n{event}\n====')
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

In [47]:
research_report = '''
## 夏向けスパイシー郷土料理レシピ調査報告書

### 1. 日本の郷土料理の調査

日本の郷土料理は、その土地の気候、風土、歴史、文化が色濃く反映された、地域固有の食文化です [[1]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHC3nyXagOBu5O6akcGQN6D4YD0xZCOHb1XL4xWa6lB3fAN0794ulF2Fmf_Du4SObYjMnwZYqV7HqK3Hw1b9IVs5xyXtV7ApN4BKU7aS5Wj2oFkRobPQbwTZKUSWQLG4TPPqRSU0hpuNsEnrGAMsMNY2AKUeJCs)。農林水産省のウェブサイト「うちの郷土料理」では、全国各地の1365種類もの郷土料理のレシピや歴史が紹介されています [[2]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHarWb6xPF5i45Sp1gFDdQBsJPSXDqAfFNELrZdJMIAewKsLam9g8bsHnT0mQqf1xPIDq8zNPEd6lE-VIhpq7tND7H6rFgn0_-suKLucINUfPcYYCJtQZaLGI-rD9URQEhnJ3-2LMdzxHQuhheoHvk53dr14MBlFsfvLr9I8AjUUnc7es_-_Q==)[[3]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGIItlebLnyz8qljE1UR34FJJLxX5jWSeYHBBE-yr7dlvae-flp-EFid_KlvjB5aNtiX9qkULfQZlFOzyPe0jRGIVczX2W5sejlMAIc7_Dmp1mRmQJOnGXA85i7A-Cj-Xu7zZPbHaUVjdy5drHajTCdaDMHwxByXtONQfZLcujoBVrWIxl70vAUaDEYZA==)[[4]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXERcqISKI8d5t7HOB7hBkx-c2QpG6ihPxAKDtXMHTwXEYAKYtJ8Gg6B8F_iMdxb7-sgoHfY5vIfMGBjH0zCsq2OH5XaueZTg1DrbIsahl4BM3Mpb6r0KLDGSQNFQSXzHSqJdy-sKJnxzRdLoOo0ULrx1PLXuQVcyzOsWCbxUs-rgNKXcg==)。

**代表的な郷土料理の例（5選）**

1.  **石狩鍋（北海道）**：鮭をぶつ切りにし、昆布だしで野菜とともに煮込んだ味噌仕立ての鍋料理です [[5]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEJoWOjPt-f8ATsZbqyMImApOs1h1eTP2JEkCFiJk1xnYf-SgvykONJhuK9isF11F9AQJjiq_KzWwleeYCEGuWAeAECsPiSNSiw7DY1K1Tmb0H8SMh6kM7kNtTq9qi9i9dxpNonBx_quy7MamLPL6X-fZCTv8nfbgbXisTOKNxw6-XoPGXABKdgUqxDH8aaSeLNE7qrC7g5HKqK2eynDvSPdEIYZnbaYcCHaA==)。
2.  **きりたんぽ鍋（秋田県）**：つぶしたご飯を竹輪のように棒に巻き付けて焼いた「きりたんぽ」を、鶏肉や野菜などと煮込んだ鍋料理です [[1]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHC3nyXagOBu5O6akcGQN6D4YD0xZCOHb1XL4xWa6lB3fAN0794ulF2Fmf_Du4SObYjMnwZYqV7HqK3Hw1b9IVs5xyXtV7ApN4BKU7aS5Wj2oFkRobPQbwTZKUSWQLG4TPPqRSU0hpuNsEnrGAMsMNY2AKUeJCs)。
3.  **冷や汁（宮崎県）**：焼いた魚をほぐしたものと味噌、薬味を混ぜてだし汁で伸ばし、冷やしてご飯にかけて食べる宮崎県の郷土料理です [[6]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE0aPy3ZDhWoWpmZQ3xc2h1ko2qj2DYOvsbe8iqrf133xqz2gIh_krOPLeKy5sX_LT95NBW8jL0cw-RbOtvaxYGNIsUBHJMwRi_tCyLwob5pvTOSYygP3xLaXwUWs_HzHTePD3gbQ8-aN4epNPE)。豆腐を入れるのが特徴で、宮崎県民のソウルフードとして親しまれています [[1]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHC3nyXagOBu5O6akcGQN6D4YD0xZCOHb1XL4xWa6lB3fAN0794ulF2Fmf_Du4SObYjMnwZYqV7HqK3Hw1b9IVs5xyXtV7ApN4BKU7aS5Wj2oFkRobPQbwTZKUSWQLG4TPPqRSU0hpuNsEnrGAMsMNY2AKUeJCs)。夏に食欲がない時でも食べやすく、家庭でも日常的に食されています [[6]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE0aPy3ZDhWoWpmZQ3xc2h1ko2qj2DYOvsbe8iqrf133xqz2gIh_krOPLeKy5sX_LT95NBW8jL0cw-RbOtvaxYGNIsUBHJMwRi_tCyLwob5pvTOSYygP3xLaXwUWs_HzHTePD3gbQ8-aN4epNPE)。地域や家庭によって様々なバリエーションがあり、アゴだしや八丁味噌を使った冷や汁も存在します [[7]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEeD1Tx2O4pXXlITJtTaQeODCDD_EypCK_LDOdtzeUwKXSMMNXRpgj73KDuH-01qW74VQVyIfe9MgvM3oYZCx4YMBvnVFluumUP8faN4mO8e6rsbs94T4fVI49ZUXyoKWjupe_esitKlG6778tH)[[8]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEFX4xmZPPi3VfWiCBGh-zC4QxqoRYfBtg-DGEd-XmjA2fg_Akx71eMer9enyTaS87VgslzUz6iYEsUQZ0xps-kONYE01JFy7d5uk2BD_6ikbAWqJvNdzwss9JO9eXHH2t6QxVc0txmkJWpZ1viBp8EK7P59qPK98nXqkfODxYsPQCHkg==)[[9]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFFRE8btzV3N15K50gPBlgbKFiQPkaKB-I78lbWTx14dSKsi1GVRaC0PgDNonfeW7-LPszZmLtPe15h5u7TuPylDkDFCYQfOXGhuagzulvoMFcLXQq8G9qZKBU9nNE5zPY8vq1DePHizuIXC9k=)。
4.  **もぶり飯（愛媛県）**：瀬戸内海の小魚でダシをとり、甘めの合わせ酢で味付けした酢飯に、季節の魚介を盛り付けたちらし寿司です。愛媛県松山の郷土料理として、祝い事やもてなしの席で食べられています [[4]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXERcqISKI8d5t7HOB7hBkx-c2QpG6ihPxAKDtXMHTwXEYAKYtJ8Gg6B8F_iMdxb7-sgoHfY5vIfMGBjH0zCsq2OH5XaueZTg1DrbIsahl4BM3Mpb6r0KLDGSQNFQSXzHSqJdy-sKJnxzRdLoOo0ULrx1PLXuQVcyzOsWCbxUs-rgNKXcg==)。
5.  **がめ煮（福岡県）**：鶏肉、ごぼう、人参、里芋、しいたけなどを甘辛く煮込んだ、福岡県博多を代表する煮物です。正月や結婚式、祭りなどの際に作られます [[1]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHC3nyXagOBu5O6akcGQN6D4YD0xZCOHb1XL4xWa6lB3fAN0794ulF2Fmf_Du4SObYjMnwZYqV7HqK3Hw1b9IVs5xyXtV7ApN4BKU7aS5Wj2oFkRobPQbwTZKUSWQLG4TPPqRSU0hpuNsEnrGAMsMNY2AKUeJCs)。

### 2. スパイシーな食材と調味料の調査

#### 2.1 夏に合うスパイシーな食材

暑い夏にスパイシーなものが食べたくなるのは、血行促進による疲労回復、発汗による打ち水効果、消化液の分泌促進による食欲増進など、暑さによる体へのダメージを回復する効果があるためです。スパイスを効かせることで食欲もアップし、夏バテせずに元気に過ごせるでしょう [[10]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFRfEM0CK2LJz_ORSBdPpnx7uejhmADYrcHclbFxsZPZLSRalhG6p28vXBzYWEAd6EM5wcNT0aNb7jLSWd5KSYS9lYAoGYoRVYgQcXwaCsrNP270WZxYhkrHJ7mxanRyPIwz5QxJdvkW_ShViJ9hLOLTcze)[[11]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEnJUbPWYn8i6djlL-tGPomv1TjaF6SJIqxFf_2PlkIN2Nqft01RX9GtUp3Bk4bNFahGNuspOhbhsO9kLdiKYY2TMqJiICn_VS-JUsfVewY4asV-zP8ECJnyDOeXeqkiI-kUhnZcST0NeWxN0ohhSs=)。

**夏に合うスパイシーな食材の例**

*   **キムチ**：夏野菜のトマトと一緒にサラダとして食べるのに適しています。
*   **空芯菜**：ニンニクや唐辛子と一緒に炒めることで、ピリ辛でおいしい料理になります [[12]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFkZTo-q48ywjrvAqVj_35_2EZ9iGKEAlo2N0KM6RzKZTAYDg-BztVtYuftUTFa9x1hPmrL6CpG967-7XYziEJa8Ui6oDak3XkJ16wFAqeuL4j8vNuaDjlVXg==)。
*   **デュカ**：中東発祥のスパイスナッツで、様々な料理に使える万能調味料です。おむすびやアボカドとバゲットに使ったり、ナスにかけたりできます [[13]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFYvb-dG4zpAmu1yQoYHWJopshn5PppUhOl1xC6OF69_ORjPChkmiDaCxYg9G0gl0Xu0g_MHtCqiRB5ItWoCRa6CZ0cIJFyYghGYftPgXDoLdctYP7Wx9qN4wdwWCJ4salEPeUTLVWCLQ==)。
*   **山椒**：柑橘系の香りとピリッとした痺れる辛さが特徴で、清涼感を与えてくれます。
*   **生姜**：体を温める効果がある一方で、清涼感のある辛味も持ち合わせており、夏バテ気味の体をシャキッとさせてくれます。
*   **ワサビ**：独特のツンとした辛さが食欲を刺激し、冷たい料理との相性が抜群です [[14]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHqTAK8CXZr5FWhCblAxRX7w7eYBOzC5om24zDz1U-OTHqTffsZyU0aq8bMHkDgMS_WPm1_hID2lXT_3wEquu2Pt-h15sk4DXjZNzRw-lUiOyYjp0gLS9YoxXeioFQYpnqhfOLY8FtHwxf217SB8Ag=)。

#### 2.2 夏に合うスパイシーな調味料

唐辛子以外にも、夏に合うスパイシーな調味料は数多く存在します。

**唐辛子以外の夏に合うスパイシーな調味料の例**

*   **ガラムマサラ**：複数のスパイスをブレンドしたミックススパイスで、いつものメニューに加えるだけでスパイシーな夏向きの味になります。オリーブオイル、ニンニクと混ぜてスパイシードレッシングにしたり、鶏肉の唐揚げの下味に使うのもおすすめです [[15]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXF8xuLaNKiDmHIh41x6bBsUL4IO22cTIYtlb4xPm_-BqHI2bC6EBfrWEs93vHNCigUVXl9FY6WP3wZXQtVfG0IWI8fQXPJe-bjq2KvGagQPjPd9gLyhehJfUU-5Ky7bz3VSR-qdsy3K)。
*   **デュカ**：中東発祥のスパイスナッツで、様々な食材で作れる万能調味料です。アボカドとバゲットに添えたり、ナスにかけたり、おむすびに混ぜたりして楽しめます [[16]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHuFmj2RF7M0G43qVENeaM8odC1v9kDhc8U3soMbb95aplUO67IM0nCpiZfLm5azXrMt9MRz92yVuX3yNzA0G3b1NUKLuEnPFUtZxvnVYHU64G4UcC4ci9US1E-cLW_cb5jEPaUaU_U5J0=)。
*   **夏のスパイスミックス**：チリペッパー、パプリカ、クミン、黒こしょう、乾燥ハーブなどを混ぜ合わせた自家製ブレンドスパイス。鶏のトマトソース煮込みやレバーの照り焼きなど、様々な料理にひと振りするだけでスパイシーな風味を加えることができます [[17]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFKWgT7sK0QTutR2Xip_DGpuYLZnVhTmDOkyR6hehGDBLafNSarqGraX9FYfgkdoE8wWTlbt0c9FrcKXbxzGNsU7ODoW8i7eWpeCxPpGPFmr0QLFXCYzzVtebsCxnNV-UKsc67b7avuk7MOVdA=)。
*   **山椒**：柑橘系の香りとピリッとした痺れる辛さが特徴で、清涼感を与えてくれます。
*   **生姜**：体を温める効果がある一方で、清涼感のある辛味も持ち合わせており、夏バテ気味の体をシャキッとさせてくれます。
*   **ワサビ**：独特のツンとした辛さが食欲を刺激し、冷たい料理との相性が抜群です。

#### 2.3 ラー油と豆板醤の違い

ラー油と豆板醤は、どちらも中華料理などでよく使われる辛味調味料ですが、原料や味に違いがあります [[18]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGABRj800JCwnEGgdpB2pgMAfpTkKcMNIlLgPC1IWru4BBda728D1k1YpB8zxvvaEpo4OuYSHEeBhYYDw39BdxQdJxwuvIruRVLTQ_rQdFHx3Epfv12U7psXmY7xL0HwUYTHI4=)。

*   **原料**：豆板醤はそら豆を発酵させて作られますが、ラー油は植物油がベースです [[18]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGABRj800JCwnEGgdpB2pgMAfpTkKcMNIlLgPC1IWru4BBda728D1k1YpB8zxvvaEpo4OuYSHEeBhYYDw39BdxQdJxwuvIruRVLTQ_rQdFHx3Epfv12U7psXmY7xL0HwUYTHI4=)[[19]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGQECrtEj5k8KPt_vubokZHmHlo2_deu9_y0LHlFKGo6GH9o9Jj0VggG8Dsz0x0j4JmDEplAOckKxuC4OBAYbqPEF1z1eaAPbfpjqK7v9r244ISxjadlq_vIm44wICeat2s_upzrQ==)。
*   **味**：豆板醤は塩味とうま味がある一方、ラー油は辛味と香りが主で、塩味やうま味はほとんどありません [[18]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGABRj800JCwnEGgdpB2pgMAfpTkKcMNIlLgPC1IWru4BBda728D1k1YpB8zxvvaEpo4OuYSHEeBhYYDw39BdxQdJxwuvIruRVLTQ_rQdFHx3Epfv12U7psXmY7xL0HwUYTHI4=)。

豆板醤の代用としてラー油を使うことはできますが、ラー油は油っぽくなる可能性があるため、量を調整する必要があります [[19]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGQECrtEj5k8KPt_vubokZHmHlo2_deu9_y0LHlFKGo6GH9o9Jj0VggG8Dsz0x0j4JmDEplAOckKxuC4OBAYbqPEF1z1eaAPbfpjqK7v9r244ISxjadlq_vIm44wICeat2s_upzrQ==)。豆板醤の代わりに使用できる調味料は他にもあり、例えば、コチュジャン、キムチの素と味噌の組み合わせ、味噌、醤油、唐辛子、ごま油の組み合わせなどが挙げられます [[18]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGABRj800JCwnEGgdpB2pgMAfpTkKcMNIlLgPC1IWru4BBda728D1k1YpB8zxvvaEpo4OuYSHEeBhYYDw39BdxQdJxwuvIruRVLTQ_rQdFHx3Epfv12U7psXmY7xL0HwUYTHI4=)[[20]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHssPBZIKTUsKyoUC-ct_UzKLqQU4WmdsC82s8czGAO97o1mF2W4izKEL1MsT0-LDxxl1t2r3_WL9SZ4q53JrgKz0o-qz8Tly1SJwXACwuG6jG4plU=)。ラー油の代わりにごま油を使用すると、風味が豊かになります [[21]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHZbjHPezduoQ7za8fA2v_TVT6eT2n4zmO1cctii_Oz098rBY4K5J0AGhBE2VW9IPsU2CoqBP8KLUDIDoh9YjyQ3N88iNhns9yOZ71eLvjUk-rvhnsOMQ==)。

### 3. 郷土料理とスパイシー食材の組み合わせの検討

#### 3.1 スパイシーなアレンジがしやすい郷土料理

夏向けの郷土料理で、スパイシーなアレンジがしやすいものとしては、以下のものが挙げられます。

*   **夏野菜のスパイシーパエリア**：パエリアはスペインのバレンシア地方の郷土料理ですが、日本のちらし寿司のように、お好みの夏野菜やシーフードをたっぷり入れて、クミンなどのスパイスで風味を加え、スパイシーにアレンジできます [[22]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHU2_qn3mQZjqGD4JX85A8XZyY3dC3Kmiwf2ZF7mUXBUil3j0VJZDe8dOR3wXmhPPptOviAg64doorzHwoGpbSI-zJ8hilWWP2OV_k2bq47PietiFOwC0B3pIMJT3hzL5iA)。
*   **山形のだし**：きゅうりやナスなどの夏野菜を細かく刻み、香味野菜や納豆昆布などと混ぜて醤油で味付けした山形の郷土料理「だし」も、お好みでラー油や豆板醤などを加えてスパイシーにアレンジできます [[23]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGldjolGepx1OLUYXTroXYi2kG7EqjwNa_qK7nhIcGFGTQxLzlEEFe0u3HInX3euxFgOADjgw_M_GU_5ypUCVoU2r9qJwRmtj_eGX-cU4LkDINgQW3dVkxrkir7f7Qcv7TCsXZK)。
*   **インドネシア料理**：ナシゴレン（焼き飯）、サテ（焼き鳥）、ソトアヤム（鶏肉のスープ）などのインドネシア料理は、もともとスパイシーな味付けで、夏にぴったりです。サンバルなどのチリソースやスパイスを加えて、さらに辛さを調整することも可能です [[24]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXF8sRtROlbUxw-V17eIXRFdvDiluDTNYC7dTrfWKJDuJXTnyjVqHulcOY-HUZ4NJwo5n6A8387vWcwX1A09FAarLS6vS1uyGAvSp9CTlz8ykEeuvfmjQ8yIECS7EW3oOw==)。
*   **夏カレー**：カレーはスパイスが効いているため、夏バテや熱中症予防にもなると言われています。夏野菜をたっぷり使ったカレーは、見た目も鮮やかで食欲をそそります。

これらの料理は、いずれも夏野菜をたっぷり使っており、スパイスや辛味を加えてアレンジすることで、暑い夏でも食欲をそそる一品になります [[25]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHTZMCgadR2qsiE9XX9ZCgKK_UyAXzfD3gu6OPjX8zP2MFMqOcgGQXuYYXxjUJN0LbZNhEIpA3jSkQbmx-4-BOXprSs5AaNn_iq4ninB5LQMb7fcl4zUNPI)。

#### 3.2 スパイシーな食材を組み合わせる際の注意点

郷土料理にスパイシーな食材を組み合わせる際の注意点について、具体的な情報源は見つかりませんでしたが、一般的なスパイシーな食材と料理の組み合わせに関する情報から、いくつか注意できる点を以下にまとめます。

*   **風味のバランス**：スパイシーな風味は強いため、郷土料理の元の風味を損なわないように、少量から加えて調整するのがおすすめです [[17]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFKWgT7sK0QTutR2Xip_DGpuYLZnVhTmDOkyR6hehGDBLafNSarqGraX9FYfgkdoE8wWTlbt0c9FrcKXbxzGNsU7ODoW8i7eWpeCxPpGPFmr0QLFXCYzzVtebsCxnNV-UKsc67b7avuk7MOVdA=)。
*   **食材の相性**：スパイシーな食材が、郷土料理に使われている他の食材と相性が良いかを確認することが大切です。例えば、ハーブのタイムは、ラム肉、鶏肉、白身魚、野菜、卵、チーズ、フルーツなど、多くの食材と相性が良いとされています [[26]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHrjeheBcUJt48D39Z6olNwysER5wUK9pBGe_bnM6Iu6CuGb3RwxUZox_6-8ty3_wWL89Ju1xlBZfRF8mX6rX0sSFeGM_hdg_wvgVYZ3tYorbWFo582cPPaiIlikX7x)。
*   **辛さの調整**：唐辛子などの辛味成分を含む食材を使う場合は、辛さのレベルを調整し、料理全体のバランスを考慮します [[27]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHxcZhvdGO33rMnEyCmO50TOe7m6fZJCxQqUYeYPpNR4KKhuu9y9nDMddYCQ_AbQatg5_Z7wFGMiihzpFDxnYslyiDysfPxlGJbw4nVYD18LyHcWnPVBJy6o9FYG3miUFfaJLqNrHiqjAFxZLTqvnUJErRwOSdygdsZ)。
*   **地元の食材との調和**：徳島県では、フィッシュカツのように、魚のすり身にカレー粉や唐辛子を加えてスパイシーな風味を加えた郷土料理があります。このように、地元の食材とスパイシーな食材を組み合わせることで、新しい味わいを生み出すことも可能です。
*   **アレルギー**：スパイシーな食材の中にはアレルギーを引き起こすものもあるため、注意が必要です [[28]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGDhnWB9grdBrsUaIOhqqFDXK4WJl3MEIypOFJho82cz4MbnKo3TKWCZq6Jd3RjnPFNfYFUv-oz5lAyBLcHmTO2IUY8myR81requAzTn5De46B6uno0TnKYWJBhcoO2fYJfvQgBaJ_6ymCzjsp644mwPYjLpXsS0Ojt)。

#### 3.3 各郷土料理に合うスパイシーな調味料

提供された情報源からは、石狩鍋、きりたんぽ鍋、冷や汁、もぶり飯、がめ煮に合う具体的なスパイシーな調味料に関する情報を得ることはできませんでした 情報源なし]。しかし、それぞれの料理に合う可能性のあるスパイスや調味料について、以下に情報源から得られた情報をまとめます。

*   **石狩鍋**：
    *   **山椒**：本場の石狩鍋の仕上げには山椒を振ると良いでしょう [[29]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXESCu2bO_QRtQ1iVMmtpTae97Y9vwrrAx-WEtzzyYPxbb-VhvxiBGBL3U0Gv35j6NKRoGqsfFXnu9AaJErwORfXGGbeCzCl77D2fdMhwneWfVGDRnmmwxEzBIm-iWlBcR7E5uYasdYNGQ==)[[30]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXF3fQDtIYXy901iT-1Cdk-RlSQvFokD-UGkPGyhmAPyHDYtqCUgFCc11n1b3LIE0P5c0RbSuEv5a9diZjdor8unnU3zP9B28G72Lj0-6OWRcRF3nobKUCtp6V5BQq8Ft6voin5-VBva1w==)。
    *   **カレーパウダー**：石狩鍋の〆にカレーパウダーを加えると、スパイシーなカレーラーメンとして楽しめます [[31]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHbrB6BnM4O50IXRfgOntoT2WznEgih3Kh7-Hc4VBF8WApr9STo4uqSF0-NC1jUQNpIkmMh-XoyVJq171fijTzy006q3mFcHLbyTx6yzSVwUCGGppk9kYQnDEVM4df4xzXDB6U2V3sd5dShXIlc4L5LqaHpEtD4)[[32]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE9r6lPBkWAXnnIDGsdnPPBgBHdyZVscJZ9L8lyCnLb9izlsg0hd-Xv0wbgsziU5Dy_0sQQSn3Oaq-Fs6ZNcGFIXWpKjEcsvElsBqIdcvOJCi-WbF3ROZzAVQ5SBZGojF7kGaEYE1ZbYM4cjfIfOojJ3v0QAr4y)。GABANカレーパウダーを使用すると、数種類のスパイスがブレンドされており、香り豊かでコク深いカレーラーメンに仕上がります [[32]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE9r6lPBkWAXnnIDGsdnPPBgBHdyZVscJZ9L8lyCnLb9izlsg0hd-Xv0wbgsziU5Dy_0sQQSn3Oaq-Fs6ZNcGFIXWpKjEcsvElsBqIdcvOJCi-WbF3ROZzAVQ5SBZGojF7kGaEYE1ZbYM4cjfIfOojJ3v0QAr4y)。
    *   **白胡椒**：ワインと合わせる場合は、白胡椒のような仄かなスパイス香を持つワインを選ぶと相性が良いでしょう [[33]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGyaQ_boPO1MUXulpEHS0-osLoNAPo_3W9mUDmP4oT8ABQgc5UYPGUu8iGBmFoP1VwO1Ql190grzyKZH6XrSaXYrqPkAFoKCpJq3U5cEB-PslenUFoyoHJEJ2Nr0AQ7aTcXsAk=)[[34]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHr15HcX516_t0mz3IJHZB2pN_UvJY9CTZ4aitrZedkH93NKBC_ibTX40N_SmckktQRf9vKzCFWXmia8R_Rgoy4jcSHcKU06hT3vVji1RzrtWSFXttQdi9IVKofG35IuCezfkY=)。
*   **きりたんぽ鍋**：
    *   **七味唐辛子**：きりたんぽ鍋専用の七味唐辛子がおすすめです。一般的な七味は唐辛子、山椒、麻の実、黒胡麻などがベースですが、クミンやガーリックも醤油ベースの鍋に合う可能性があります。
    *   **紫しぼり大根汁**：秋田県鹿角市八幡平の松館集落で栽培されている希少な紫しぼり大根は、独特の風味と強い辛味が特徴で、きりたんぽ鍋に合う薬味として紹介されています [[35]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXErLaJbcBmzGdtGe7MZEfVwiG1ZsDVOO1F1kplflqQVojjFUlJSWoWpsez6y5GCklAjNz8rtzZC5xxIGeGteZffVHdp_6P5SyvY_TUMQQXIwvFSO_CdDvoMX8cNZ4u2K5NHxnRR7IiEejsCHDc_-1hacogtevn0y4P7WP88oB-_k9ltEkTn-opkVT-aSkEQS6BNlkBoUlGYFKxC4pGpyXXSgPM9rL7LxsyFO4XPDqxYKWpYzlIQRWkAn-Pwy16IP5U9f62ioObv0YcGQalYBtnqDd_x6mg6DUuc3X7gSRi7anlDYYDVjJdvMBDgVmuTDSJsOXiZWJIf0ZpmDk8uL5xIBMVOklZffzsUlb3JVKsdp5xa6_8s9u184IlIUAydFOekdq4PtIg=)。
    *   **S&Bのラー油、カレー粉、クミンシード、おろししょうが**：きりたんぽカレー鍋のレシピでは、これらのスパイスが使用されています。ラー油はアクセントになり、クミンシードとおろししょうがは風味を加えます [[36]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEIn09Gwz9FLhTULa4Xk87Lf8qPoclOJ8w_Tn1d3_VEU2EP241S9Mf_yTxaSb66YO--dPdzojpt1GCZU24UcZOybG51391vvUi1yAL5PDKqiftORYk5tOONseFn93tk0z5w83Wl4uFiCEsa23w3)。
    *   **山椒塩**：吉井南美さんのレシピでは、山椒塩がスパイス鍋に合うと紹介されています [[37]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEemEEffc1UKWp66J9FZcDSq3k6urMBrjp8rKl-i4RppeLXc49ASak80AomCAcK0c58T08u0u5ZPRB2T4gXqb3Y3wlZ6AaJKIY9Ae0ElPUU4z4tAza4CCrIcLi1xL3OhNdbYi0xJzF_p81XPhSNKdK9vtuVKkSZUkEJ-1zeyMA9BGu-)。
*   **冷や汁**：
    *   **ショウガ（ジンジャー）**：さわやかな香りと辛味が風味付けや薬味として使えます。
    *   **シソ**：清涼感のある香りが和風料理によく合います [[7]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEeD1Tx2O4pXXlITJtTaQeODCDD_EypCK_LDOdtzeUwKXSMMNXRpgj73KDuH-01qW74VQVyIfe9MgvM3oYZCx4YMBvnVFluumUP8faN4mO8e6rsbs94T4fVI49ZUXyoKWjupe_esitKlG6778tH)。
    *   **ワサビ**：鼻に抜けるツーンとした辛みが特徴です [[6]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE0aPy3ZDhWoWpmZQ3xc2h1ko2qj2DYOvsbe8iqrf133xqz2gIh_krOPLeKy5sX_LT95NBW8jL0cw-RbOtvaxYGNIsUBHJMwRi_tCyLwob5pvTOSYygP3xLaXwUWs_HzHTePD3gbQ8-aN4epNPE)。
    *   **ミョウガ**：きざみミョウガは冷や汁によく合う薬味として紹介されています [[38]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHVMeT4a09fe3EaPOo3lqgRvHKcSAseCihDL2iSt5y3m0t03rDHz4FFQB3wJCwfVsUbrAQUgoCY0f9v55YaOSkCkLydNVGgYmnEyblM-c81VFVenTLoMXinMf0rdqjg6yvRXYcervDZHRq8F4kK)[[39]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHBpfAUc3ZwldVftQEZ61-e-cVecZeq5EbCIQ6rY2WrWEz16gZDdRLgP8YFNF1xEaAoIqLz8DPuLK8B0BndsIG_dMoI8rvhA-6stx6W7MFDGjrHAI8nr9JW8R7z6PbebGiNe3pW6YvDoUh6uibVwE3rz-UCQlSrevvmMBYJqLPKho2VEA==)。
    *   **ゴマ**：すりごまや煎りゴマは、冷や汁の風味を豊かにするために使われます [[38]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHVMeT4a09fe3EaPOo3lqgRvHKcSAseCihDL2iSt5y3m0t03rDHz4FFQB3wJCwfVsUbrAQUgoCY0f9v55YaOSkCkLydNVGgYmnEyblM-c81VFVenTLoMXinMf0rdqjg6yvRXYcervDZHRq8F4kK)[[39]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHBpfAUc3ZwldVftQEZ61-e-cVecZeq5EbCIQ6rY2WrWEz16gZDdRLgP8YFNF1xEaAoIqLz8DPuLK8B0BndsIG_dMoI8rvhA-6stx6W7MFDGjrHAI8nr9JW8R7z6PbebGiNe3pW6YvDoUh6uibVwE3rz-UCQlSrevvmMBYJqLPKho2VEA==)[[40]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFE9t5YTlmpTOfeQ8-z2ppr6b1BbJMXdmrJyhL-HoGW7OKHT2aG44bvk14m9XYeFEPymUOyy8ceF-TJjQ-Gecs3TDkoweJUe30zrpyYlsOgsnvTV-I60iwxy_Z8x90jANvvR9SNFDHdcL25O48NcZVbsh9yZXPKf4Vz1EuDq4Gr)。
    *   **和山椒**：山椒は香りと辛みが特徴で、冷や汁にアクセントを加えることができます [[38]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHVMeT4a09fe3EaPOo3lqgRvHKcSAseCihDL2iSt5y3m0t03rDHz4FFQB3wJCwfVsUbrAQUgoCY0f9v55YaOSkCkLydNVGgYmnEyblM-c81VFVenTLoMXinMf0rdqjg6yvRXYcervDZHRq8F4kK)。
    *   **ラー油**：エスニック風にアレンジしたトマト冷や汁にラー油をかけるレシピもあります [[41]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHFRdWc6OWnxSiHQJ1X2wQhDyTQw2hTDEeyLXnCWkA0sL4MlPcMgCjIo_zeNmseH7q7BVmhSf8-8-w-Alh1bFqkT3hP1SwuHmHHUoQLf4BVye9n0NaQNmRGMBxo1oLucyXiE95lGfM=)。
*   **もぶり飯**：情報源から具体的な提案は見つかりませんでした 情報源なし]。
*   **がめ煮**：情報源から具体的な提案は見つかりませんでした 情報源なし]。

#### 3.4 各郷土料理に合う夏野菜

*   **石狩鍋**：情報源から直接的な回答を得ることはできませんでしたが、石狩鍋は一般的に、鮭、野菜、豆腐などを味噌で煮込んだ鍋料理であり、レシピには様々な野菜が使われています [[42]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGtAHzFz5nsfSXJ2r6hzBwZXv29I92FdQmaBJkEXskHnVEdVNvH5Eey49sfQSI3ZZ3hZLKcM2r3_0Pv7dh6T-02VViHeYESw5kF3OONOcrFRE9r7k1Cucyg8xYrAWOdqd2iKTrCPM36h9qVLca5WePopEju)[[43]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXF73BXdt7ik9CTxCfBqjApb-c5kRbiQYMaWHsbyp1mzKhCfEwrIKFfw2GSka08nJYGM8NIE3Fhuj7SGiUuz0GEaMPHj-m2xzw7IJzqki_-kS6wDQL2cnz9x4yxR5Mp8tt8=)[[44]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHlKL_eO0o8pw-QOPTKBxqB0N6e0-5EZ3cCZeilKRIJKDu8A0OwUUupeC8DgtqPpdc5Hxna0riMGmcb-M-Y2L95FQedivrEJvnJF-j8xUrSeBI5v19PEz--PQGC8sjfJ0Ir4qSjy1YkqVW2OQXu)[[45]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHXYN3IjJzgh-6_N_s7lvITxeL85lz2WdIkKYtgjOvxrgTwkNJQOJhGG3GqfDAYWlDYXQth1ptE7FkGpRSeV7Nt6x0ifUMD3cvUT9M0YUbTdaaFOA-ucIcloPlIjXcEeImD6B13CYgnbyxKTOTnxvsPIl78NffpvKJdK73q0R2BMd1O1V6TOA==)[[46]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGYwNb4FrXa9DvhP4T-7fuoitv-rJQGsQHqIhDbxldbOSiBQOKU7YiESEK74SFVwkbdXCG2_5tW_4i1TkIgBYjI3MTRLrypY82VoK46QzS1cj8Xwbti1Zo31vBJSyx_oIt3CWi2fcHeEKAZtgCwYPw0xUywoBZAYg==)。一般的に石狩鍋に使われる野菜は以下の通りです [[42]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGtAHzFz5nsfSXJ2r6hzBwZXv29I92FdQmaBJkEXskHnVEdVNvH5Eey49sfQSI3ZZ3hZLKcM2r3_0Pv7dh6T-02VViHeYESw5kF3OONOcrFRE9r7k1Cucyg8xYrAWOdqd2iKTrCPM36h9qVLca5WePopEju)[[44]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHlKL_eO0o8pw-QOPTKBxqB0N6e0-5EZ3cCZeilKRIJKDu8A0OwUUupeC8DgtqPpdc5Hxna0riMGmcb-M-Y2L95FQedivrEJvnJF-j8xUrSeBI5v19PEz--PQGC8sjfJ0Ir4qSjy1YkqVW2OQXu)[[45]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHXYN3IjJzgh-6_N_s7lvITxeL85lz2WdIkKYtgjOvxrgTwkNJQOJhGG3GqfDAYWlDYXQth1ptE7FkGpRSeV7Nt6x0ifUMD3cvUT9M0YUbTdaaFOA-ucIcloPlIjXcEeImD6B13CYgnbyxKTOTnxvsPIl78NffpvKJdK73q0R2BMd1O1V6TOA==)[[43]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXF73BXdt7ik9CTxCfBqjApb-c5kRbiQYMaWHsbyp1mzKhCfEwrIKFfw2GSka08nJYGM8NIE3Fhuj7SGiUuz0GEaMPHj-m2xzw7IJzqki_-kS6wDQL2cnz9x4yxR5Mp8tt8=)[[46]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGYwNb4FrXa9DvhP4T-7fuoitv-rJQGsQHqIhDbxldbOSiBQOKU7YiESEK74SFVwkbdXCG2_5tW_4i1TkIgBYjI3MTRLrypY82VoK46QzS1cj8Xwbti1Zo31vBJSyx_oIt3CWi2fcHeEKAZtgCwYPw0xUywoBZAYg==)。
    *   大根 [[42]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGtAHzFz5nsfSXJ2r6hzBwZXv29I92FdQmaBJkEXskHnVEdVNvH5Eey49sfQSI3ZZ3hZLKcM2r3_0Pv7dh6T-02VViHeYESw5kF3OONOcrFRE9r7k1Cucyg8xYrAWOdqd2iKTrCPM36h9qVLca5WePopEju)[[44]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHlKL_eO0o8pw-QOPTKBxqB0N6e0-5EZ3cCZeilKRIJKDu8A0OwUUupeC8DgtqPpdc5Hxna0riMGmcb-M-Y2L95FQedivrEJvnJF-j8xUrSeBI5v19PEz--PQGC8sjfJ0Ir4qSjy1YkqVW2OQXu)[[45]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHXYN3IjJzgh-6_N_s7lvITxeL85lz2WdIkKYtgjOvxrgTwkNJQOJhGG3GqfDAYWlDYXQth1ptE7FkGpRSeV7Nt6x0ifUMD3cvUT9M0YUbTdaaFOA-ucIcloPlIjXcEeImD6B13CYgnbyxKTOTnxvsPIl78NffpvKJdK73q0R2BMd1O1V6TOA==)
    *   玉ねぎ [[43]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXF73BXdt7ik9CTxCfBqjApb-c5kRbiQYMaWHsbyp1mzKhCfEwrIKFfw2GSka08nJYGM8NIE3Fhuj7SGiUuz0GEaMPHj-m2xzw7IJzqki_-kS6wDQL2cnz9x4yxR5Mp8tt8=)
    *   長ネギ [[42]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGtAHzFz5nsfSXJ2r6hzBwZXv29I92FdQmaBJkEXskHnVEdVNvH5Eey49sfQSI3ZZ3hZLKcM2r3_0Pv7dh6T-02VViHeYESw5kF3OONOcrFRE9r7k1Cucyg8xYrAWOdqd2iKTrCPM36h9qVLca5WePopEju)[[45]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHXYN3IjJzgh-6_N_s7lvITxeL85lz2WdIkKYtgjOvxrgTwkNJQOJhGG3GqfDAYWlDYXQth1ptE7FkGpRSeV7Nt6x0ifUMD3cvUT9M0YUbTdaaFOA-ucIcloPlIjXcEeImD6B13CYgnbyxKTOTnxvsPIl78NffpvKJdK73q0R2BMd1O1V6TOA==)
    *   人参 [[42]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGtAHzFz5nsfSXJ2r6hzBwZXv29I92FdQmaBJkEXskHnVEdVNvH5Eey49sfQSI3ZZ3hZLKcM2r3_0Pv7dh6T-02VViHeYESw5kF3OONOcrFRE9r7k1Cucyg8xYrAWOdqd2iKTrCPM36h9qVLca5WePopEju)[[44]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHlKL_eO0o8pw-QOPTKBxqB0N6e0-5EZ3cCZeilKRIJKDu8A0OwUUupeC8DgtqPpdc5Hxna0riMGmcb-M-Y2L95FQedivrEJvnJF-j8xUrSeBI5v19PEz--PQGC8sjfJ0Ir4qSjy1YkqVW2OQXu)
    *   キャベツ [[42]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGtAHzFz5nsfSXJ2r6hzBwZXv29I92FdQmaBJkEXskHnVEdVNvH5Eey49sfQSI3ZZ3hZLKcM2r3_0Pv7dh6T-02VViHeYESw5kF3OONOcrFRE9r7k1Cucyg8xYrAWOdqd2iKTrCPM36h9qVLca5WePopEju)[[44]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHlKL_eO0o8pw-QOPTKBxqB0N6e0-5EZ3cCZeilKRIJKDu8A0OwUUupeC8DgtqPpdc5Hxna0riMGmcb-M-Y2L95FQedivrEJvnJF-j8xUrSeBI5v19PEz--PQGC8sjfJ0Ir4qSjy1YkqVW2OQXu)[[45]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHXYN3IjJzgh-6_N_s7lvITxeL85lz2WdIkKYtgjOvxrgTwkNJQOJhGG3GqfDAYWlDYXQth1ptE7FkGpRSeV7Nt6x0ifUMD3cvUT9M0YUbTdaaFOA-ucIcloPlIjXcEeImD6B13CYgnbyxKTOTnxvsPIl78NffpvKJdK73q0R2BMd1O1V6TOA==)
    *   じゃがいも [[42]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGtAHzFz5nsfSXJ2r6hzBwZXv29I92FdQmaBJkEXskHnVEdVNvH5Eey49sfQSI3ZZ3hZLKcM2r3_0Pv7dh6T-02VViHeYESw5kF3OONOcrFRE9r7k1Cucyg8xYrAWOdqd2iKTrCPM36h9qVLca5WePopEju)[[46]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGYwNb4FrXa9DvhP4T-7fuoitv-rJQGsQHqIhDbxldbOSiBQOKU7YiESEK74SFVwkbdXCG2_5tW_4i1TkIgBYjI3MTRLrypY82VoK46QzS1cj8Xwbti1Zo31vBJSyx_oIt3CWi2fcHeEKAZtgCwYPw0xUywoBZAYg==)
    *   小松菜 [[42]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGtAHzFz5nsfSXJ2r6hzBwZXv29I92FdQmaBJkEXskHnVEdVNvH5Eey49sfQSI3ZZ3hZLKcM2r3_0Pv7dh6T-02VViHeYESw5kF3OONOcrFRE9r7k1Cucyg8xYrAWOdqd2iKTrCPM36h9qVLca5WePopEju)[[44]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHlKL_eO0o8pw-QOPTKBxqB0N6e0-5EZ3cCZeilKRIJKDu8A0OwUUupeC8DgtqPpdc5Hxna0riMGmcb-M-Y2L95FQedivrEJvnJF-j8xUrSeBI5v19PEz--PQGC8sjfJ0Ir4qSjy1YkqVW2OQXu)[[45]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHXYN3IjJzgh-6_N_s7lvITxeL85lz2WdIkKYtgjOvxrgTwkNJQOJhGG3GqfDAYWlDYXQth1ptE7FkGpRSeV7Nt6x0ifUMD3cvUT9M0YUbTdaaFOA-ucIcloPlIjXcEeImD6B13CYgnbyxKTOTnxvsPIl78NffpvKJdK73q0R2BMd1O1V6TOA==)
    *   しいたけ [[42]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGtAHzFz5nsfSXJ2r6hzBwZXv29I92FdQmaBJkEXskHnVEdVNvH5Eey49sfQSI3ZZ3hZLKcM2r3_0Pv7dh6T-02VViHeYESw5kF3OONOcrFRE9r7k1Cucyg8xYrAWOdqd2iKTrCPM36h9qVLca5WePopEju)
    *   ピーマン
    *   ナス
    これらの野菜に加えて、旬の夏野菜であるカボチャやトウモロコシなども石狩鍋に合うと考えられます。お好みに合わせて、色々な夏野菜を試してみてはいかがでしょうか [[46]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGYwNb4FrXa9DvhP4T-7fuoitv-rJQGsQHqIhDbxldbOSiBQOKU7YiESEK74SFVwkbdXCG2_5tW_4i1TkIgBYjI3MTRLrypY82VoK46QzS1cj8Xwbti1Zo31vBJSyx_oIt3CWi2fcHeEKAZtgCwYPw0xUywoBZAYg==)。
*   **きりたんぽ鍋**：情報源からは具体的な提案を見つけることができませんでした 情報源なし]。情報源には、きりたんぽ鍋の一般的な材料やレシピ、アレンジに関する記述がありますが、特に夏野菜に焦点を当てたものはありません [[47]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXF7EL5qOC4O9_jC5RScsJ7uAJCpm456v7I9m41WnGJc78ODYfdaSKpLtiNNDzD_yTu44e8P_y8veB4ftFbxF-QvSqxaYA6yvqxeD4KughQ5xXwaw77zAjPeuousfjyYHTeBd9BbwjhXxDbQng==)[[48]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEhR7yZ5N7nEHVS-0Y7ya3feGiMmqilkVZZXdSF7s2bcim3GRboouN7ArATzeitva1MfmgBpI2YkA6FV-FOUj1fJi_kDmdbogcl76iu89z2DufVC_kUkI0OzCj4V80hThQ6K_F9udpjOAW-MmjrCjemI27Fpzs0gas=)[[49]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHeZdpnDpvCyVwwyHyl4qR6qGtG2KcHPi4-HX05DjRQb5RomhbVzjiczGyklIWqzX5UiVlAW7PHu7x88BTqy59_Gk_eLkcqbUFsjzBx5lZCJ9qzJC9ov6-d96tdD4O-tQyVmcmMw67pk_k7Xxld1aXcnxIF0kZ33gFdMw==)[[50]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGGYEvDfHEvJbPYlEj9dBGR43TeCg2F8cm2fIMI8AGC9Cty5n3BLMddBkAnoayB3BNR8awSSM6SZRD1rhkj7nqcZDN6n2bynyP7TJa53_hM8LhvMCRm58khBNFCWQK9o-7gnHrvOiPU)[[51]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE2V02M-9qEb-Le9em10zfa_5LoMTUDcj0mx1PFUvl59VuUiGVknvtqiV6Rag_ApTR600qBerX0ih-_CEsGwgy9TV26O1ZJ-7NfLyLR0TTEINSHkPWvnRRoE-jA5zRHriXcpAcJ)。一般的に、鍋料理には季節の野菜を自由に入れることができます。きりたんぽ鍋に夏野菜を取り入れる際は、鍋に合うアクの少ない野菜を選ぶと良いでしょう。例えば、ナス、キュウリ、トマト、オクラ、ピーマン、カボチャ、トウモロコシなどが考えられます。これらの野菜をきりたんぽ鍋に加えることで、季節感を取り入れつつ、栄養バランスも向上させることができます [[50]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGGYEvDfHEvJbPYlEj9dBGR43TeCg2F8cm2fIMI8AGC9Cty5n3BLMddBkAnoayB3BNR8awSSM6SZRD1rhkj7nqcZDN6n2bynyP7TJa53_hM8LhvMCRm58khBNFCWQK9o-7gnHrvOiPU)。
*   **冷や汁**：冷や汁に合う夏野菜はいくつかあります [[52]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGB3TJo_DP37Bd4cRhUaTRHOAP82crUOZW2pwGTSb5QJ8s8kY6SjwgIFI6wrqJqm6ZAiMLltHmHMxx-M-0GUAsGE6wqhlcKJWgPTHN1ealaTj-174gz-xQoeHrn-r55PBZh29Xf)[[53]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHQ2ZMwojSLfSNzWlsbyOiDfE3RCEEgZKDDog-rEFuWxI7AyJUYLlsu5izKpXkNyiqefkTc_IEpRGpCF4-p1rOLR0YEtGQ1cu0K6JzDkDV8GQQvDkD9aNgxjcXXeTPnf3_dsRGhLtkWHvCK)[[54]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXELeRzvbyEMjBzfJ34a_UbvFCPsMp41f62kG54n2x_h7fOqzLpPjEkv4P0bJJQkhySozXzDLX0HjbFCYWAP_8wmS7x6GYRjxsB7XhPnUlrjrFEHO3TwtYkllQCYQdd7LSuJo_pPHyfBLRFUvCupbXFL9WtP)[[55]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEuma-4uxjVIRylvFqJvqEpi7kA5xcl0qloqAoiIJ3tdTatn5vKrVjdEfSiUH6TO11owdXXdw8m269xS1TVMHUGIelDupT8WH43I-ZHdFPaX97my_Kj9BgqVo1Y75hx62gCRO7rFbQKHY5FvNM5e4PejP6HyCzH4QgDJ1ytEWItNcCBous8qrl2IJ6esNtuZA7uCh4PpSfgSi5_OuHgFywtauoxx-KTlxastqrLY1gJ9djJT-Q=)。
    *   きゅうり [[52]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGB3TJo_DP37Bd4cRhUaTRHOAP82crUOZW2pwGTSb5QJ8s8kY6SjwgIFI6wrqJqm6ZAiMLltHmHMxx-M-0GUAsGE6wqhlcKJWgPTHN1ealaTj-174gz-xQoeHrn-r55PBZh29Xf)[[53]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHQ2ZMwojSLfSNzWlsbyOiDfE3RCEEgZKDDog-rEFuWxI7AyJUYLlsu5izKpXkNyiqefkTc_IEpRGpCF4-p1rOLR0YEtGQ1cu0K6JzDkDV8GQQvDkD9aNgxjcXXeTPnf3_dsRGhLtkWHvCK)
    *   なす [[54]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXELeRzvbyEMjBzfJ34a_UbvFCPsMp41f62kG54n2x_h7fOqzLpPjEkv4P0bJJQkhySozXzDLX0HjbFCYWAP_8wmS7x6GYRjxsB7XhPnUlrjrFEHO3TwtYkllQCYQdd7LSuJo_pPHyfBLRFUvCupbXFL9WtP)[[53]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHQ2ZMwojSLfSNzWlsbyOiDfE3RCEEgZKDDog-rEFuWxI7AyJUYLlsu5izKpXkNyiqefkTc_IEpRGpCF4-p1rOLR0YEtGQ1cu0K6JzDkDV8GQQvDkD9aNgxjcXXeTPnf3_dsRGhLtkWHvCK)
    *   みょうが [[52]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGB3TJo_DP37Bd4cRhUaTRHOAP82crUOZW2pwGTSb5QJ8s8kY6SjwgIFI6wrqJqm6ZAiMLltHmHMxx-M-0GUAsGE6wqhlcKJWgPTHN1ealaTj-174gz-xQoeHrn-r55PBZh29Xf)[[53]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHQ2ZMwojSLfSNzWlsbyOiDfE3RCEEgZKDDog-rEFuWxI7AyJUYLlsu5izKpXkNyiqefkTc_IEpRGpCF4-p1rOLR0YEtGQ1cu0K6JzDkDV8GQQvDkD9aNgxjcXXeTPnf3_dsRGhLtkWHvCK)
    *   青じそ [[52]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGB3TJo_DP37Bd4cRhUaTRHOAP82crUOZW2pwGTSb5QJ8s8kY6SjwgIFI6wrqJqm6ZAiMLltHmHMxx-M-0GUAsGE6wqhlcKJWgPTHN1ealaTj-174gz-xQoeHrn-r55PBZh29Xf)
    *   かぼちゃ
    *   オクラ [[54]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXELeRzvbyEMjBzfJ34a_UbvFCPsMp41f62kG54n2x_h7fOqzLpPjEkv4P0bJJQkhySozXzDLX0HjbFCYWAP_8wmS7x6GYRjxsB7XhPnUlrjrFEHO3TwtYkllQCYQdd7LSuJo_pPHyfBLRFUvCupbXFL9WtP)
    *   ゴーヤ
    *   ズッキーニ
    これらの野菜を参考に、お好みの冷や汁を作ってみてください [[55]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEuma-4uxjVIRylvFqJvqEpi7kA5xcl0qloqAoiIJ3tdTatn5vKrVjdEfSiUH6TO11owdXXdw8m269xS1TVMHUGIelDupT8WH43I-ZHdFPaX97my_Kj9BgqVo1Y75hx62gCRO7rFbQKHY5FvNM5e4PejP6HyCzH4QgDJ1ytEWItNcCBous8qrl2IJ6esNtuZA7uCh4PpSfgSi5_OuHgFywtauoxx-KTlxastqrLY1gJ9djJT-Q=)。
*   **もぶり飯**：情報源から具体的な提案は見つかりませんでした 情報源なし]。
*   **がめ煮**：情報源から具体的な提案は見つかりませんでした 情報源なし]。

#### 3.5 和食 generally に合うスパイスの組み合わせ例

和食では、料理に香りづけ、臭み消し、風味や見た目を良くするためにスパイスが使われます。日本では、スパイスは茎、葉、花を除く植物部位と定義され、ハーブと区別されます。唐辛子、胡椒、生姜、わさびなどもスパイスに分類されます。和食に欠かせないスパイスとして、山椒、生姜、わさび、柚子、唐辛子が挙げられます [[56]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXG01EFlG38EvBLAaYUBz0K32C1W0OBHkA578c69OoQzvjCy_TktOBlZITI_rtVWDQbMXrb5gVUiL2EpNHHIr3yKbT2p7KwfCg3t0chL4-96V3wrfyn-2kLZktPzbKkbuaej2lDP4viRt_GwMFeJGEYeK5WY-59eVEeDNKSD)。

**料理別のスパイスの組み合わせ例**

*   **刺身**：わさび、生姜、にんにく、山椒（果皮、木の芽）、しそ（葉、穂）、からし、ミョウガ、たで、浜防風。
*   **煮物**：生姜、にんにく、唐辛子、山椒、しそ、からし、ゆず、くちなしの実、三つ葉。
*   **焼き物**：わさび、生姜、唐辛子、山椒、しそ、ゆず、けしの実、ごま、かぼす、すだち。
*   **和え物・揚げ物**：わさび、生姜、山椒、しそ、ごま、かぼす、すだち。
*   **汁物**：春は木の芽、穂じそ、三つ葉。夏はしそ、ミョウガ、青ゆず、実山椒。秋はゆず、ミョウガ、しその実。冬はせり、すだち [[57]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGR32TAjwW38CgQEkjVwIfe_xrUf5lkHPIyK_QKFCM1CwZ9Cqinv2pt_QSz1D1ZlHqEzrmjafzZcUt8WviWB0qALh6y54SbhWv-FUf9FZoQqLUV7S1dEeCI_k8pGtEWbrpu5f51OnsY8efc7qV8dZP7cA==)。
*   **肉料理**：
    *   牛肉：ローズマリー（グリル、ロースト）、タイム（煮込み）、オレガノ（タコス、ビーフシチュー）。
    *   豚肉：オールスパイス（煮込み、マリネ）、セージ（脂との相性良）、マジョラム（ソーセージ、グリル）、パプリカ（ロースト、焼き料理） [[58]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXErdLpxh4J4Qukhad5a-ORseOc7aJHEyK1OIPK7TmMOZGEbuaWEFRNFLSOFYwT_IWg7z30c1hO1FNjf-ljpw_yLT1FWdFZoAUnTUtFgu0KJrnpj_wNATn-mKyONXlFFzMUQYfQjk1qF2FEhupj_uzY=)。
    *   鶏肉：ローレル、ジンジャー、シナモン（煮込み）、バジル、マスタード、タラゴン（焼く・炒める）、パプリカ、タイム（揚げる）、チリーペッパー、わさび（仕上げ）。
*   **魚料理**：
    *   青身魚（イワシ、サバ、アジ）：生臭みを消す効果のあるスパイス [[59]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHoxIBl-AKEAUiJS4XKHLQxr_S9yDxU4_mMerBMo-_D3UtAcULX9qH7mJ-tBeoQNT0sqVlZpp5dMAMkpxJ1A8Xehj1q3xSfP-XqGqWYJt0lzUOeb2G0dsHMO9aubnhAqSnVSwX1Wz129Rw=)。
    *   カルパッチョ： [[58]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXErdLpxh4J4Qukhad5a-ORseOc7aJHEyK1OIPK7TmMOZGEbuaWEFRNFLSOFYwT_IWg7z30c1hO1FNjf-ljpw_yLT1FWdFZoAUnTUtFgu0KJrnpj_wNATn-mKyONXlFFzMUQYfQjk1qF2FEhupj_uzY=)。
*   **その他**：
    *   ハンバーグ：ナツメグ。
    *   カレー：シナモン、ターメリック [[60]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEEu80wm_QMcc4iXcQtjkWlfwx9z3LCYPx6FZA-JgMRUoakUy360uqtMo49jQM_4h6CK5YBvpHtRZx4YYmhS2nIDiac98SDR0_7aTQzddOq8ZVCy6sxcMs6HQaVMQv_9jF0YluaiIZjV2dHK0xjajAvHMbo)[[58]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXErdLpxh4J4Qukhad5a-ORseOc7aJHEyK1OIPK7TmMOZGEbuaWEFRNFLSOFYwT_IWg7z30c1hO1FNjf-ljpw_yLT1FWdFZoAUnTUtFgu0KJrnpj_wNATn-mKyONXlFFzMUQYfQjk1qF2FEhupj_uzY=)。
    *   ポテト料理、スープ：ナツメグ。

これらの組み合わせを参考に、お好みの郷土料理に合うスパイスを見つけてみてください [[58]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXErdLpxh4J4Qukhad5a-ORseOc7aJHEyK1OIPK7TmMOZGEbuaWEFRNFLSOFYwT_IWg7z30c1hO1FNjf-ljpw_yLT1FWdFZoAUnTUtFgu0KJrnpj_wNATn-mKyONXlFFzMUQYfQjk1qF2FEhupj_uzY=)。

### 4. レシピの考案と試作

#### 4.1 各郷土料理のレシピ

提供された情報源からは、石狩鍋、きりたんぽ鍋、冷や汁、がめ煮のレシピは見つかりましたが、もぶり飯のレシピは見つかりませんでした 情報源なし]。

**石狩鍋**

*   **レシピ1**：
    1.  鍋に水と昆布を入れ15分程度置く。
    2.  生鮭は2～3等分に切る。
    3.  鮭のアラがあれば熱湯をかけて霜降りにして洗い、水気を拭く。
    4.  1の鍋に3を加え、酒、タマネギ、ジャガイモも加えて火にかける。
    5.  途中アクを除き、ふつふつしてきたら昆布を取り出す。
    6.  再びふつふつしてきたらキャベツ、豆腐、シイタケを加えて火が通るまで煮る。
    7.  味をみて薄ければ味噌（分量外）を足し、仕上げにバターをのせる。お好みで粉山椒をふって頂く [[61]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFEIB7t2tgFbcsx4AuzGOieBLZaW_Nc1PM9wVDJgtqjbHkrifPQATT-RGld7QvDqZ3RN2Lx1iL6Hl7RC09AZe7Xmvmlixrf3AKf5pKcMVZbyX6xhmgnyOYEEngQMQwD7j2yuDft-v4144q4MSEere3U8WyRb7GAl76SmI16hcshTZm1N7wNbg==)。
*   **レシピ2（デリッシュキッチン）**：
    1.  材料（4人分）：生鮭4切れ、キャベツ1/4個、春菊1袋、玉ねぎ1/2個、しいたけ8個、木綿豆腐1丁、しらたき1袋、有塩バター20g。
    2.  スープ：水800cc、酒大さじ3、和風顆粒だし小さじ1、みりん大さじ2、みそ大さじ4。
    3.  生鮭は一口大に切る。キャベツ、春菊、玉ねぎ、しいたけ、豆腐も食べやすい大きさに切る。しらたきは食べやすい長さに切る。
    4.  鍋に味噌以外のスープの材料を入れ中火で熱し、煮立ったら生鮭を入れ、再び煮立ったらアクを取り除く。
    5.  春菊以外の残りの具材を加えて火が通るまで加熱する。味噌を溶き入れ、春菊を加えてさっと煮て仕上げにバターを加える [[62]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXF5zvsKuNz3msqOx4JnasqJq5_OqwG40uYaqO56x-UJZ_eVrOQ2GaN0cFGaFTDKD_SubJuo0X-II5zwOkVX4VD4gqwTUz8KvWN4fnrHGrgEYtSrFEajYOoIt9bGiWS3i2qjJs9ZklURVKmOd_7BLw==)。

**きりたんぽ鍋**

*   **レシピ**：
    1.  きりたんぽは斜め半分に切る。せりは根元を切り落とし、食べやすい大きさに切る。
    2.  鶏肉は食べやすい大きさに切る [[63]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHWXrNu2HtY-wcNkFB83R0KVD5eXDiuYesKekG6ZP4U6MpgHutJKZ_-J0ntOgoWGRheCFB6kluL98tlPDJvDsQgAXfQxhfiEZZth0kMMO_bi9mExDIi8wsvbvqE19aNcjHgmB6cKyE7Bp1mfgnarQ==)。
    2.  鶏肉は食べやすい大きさに切る [[64]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEbH6nZ9zYJEWz7TX3_mzz5BtYobXfPVftPL1qtAj3zd0JXYxLnMlfcM2sr0ZN-ybgl76X1lgeCraqviJxoKNzrwCnZq_VQRJj86xrfmpD5VEmG5SfiTElvsPAKjYSdXDEQxgfyTY35zpqbsk97nA==)。
    3.  ボウルに鍋つゆの材料を入れ混ぜる。
    4.  鍋に鍋つゆを適量入れ中火で煮立たせ、鶏肉を入れ、肉の色が変わるまで煮る。
    5.  ねぎ、せりを加えてしんなりするまで煮る [[63]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHWXrNu2HtY-wcNkFB83R0KVD5eXDiuYesKekG6ZP4U6MpgHutJKZ_-J0ntOgoWGRheCFB6kluL98tlPDJvDsQgAXfQxhfiEZZth0kMMO_bi9mExDIi8wsvbvqE19aNcjHgmB6cKyE7Bp1mfgnarQ==)。

**冷や汁**

*   **レシピ**：
    1.  アジを3~4分焼く。
    2.  焼いたアジの身をほぐす。
    3.  アジのすり身と味噌を弱火で炒る。
    4.  味噌がかたまってきたら火を止める。
    5.  冷たいだし汁で焼き味噌を溶かす。
    6.  きゅうり、大葉、豆腐を加える。
    7.  最後にすりごまを入れてひと混ぜする。
    冷や汁の素となる焼き味噌は、冷蔵保存や冷凍保存が可能です [[65]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHPhROtinBFQyuZPI8TzlciYZbgwwMA03C0vMvKeCpLcnKX9NrgIMGl9u0-ej0-5jbEKkt9np_KtSPt7V3ASEDDZyScots4P6IlUoinJ3FYHc2wSUE8MxEyq5uM0nZ4JObJhlEbTvE=)。

**がめ煮**

*   **レシピ**：
    1.  ボウルに干ししいたけ、水を入れて戻す [[64]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEbH6nZ9zYJEWz7TX3_mzz5BtYobXfPVftPL1qtAj3zd0JXYxLnMlfcM2sr0ZN-ybgl76X1lgeCraqviJxoKNzrwCnZq_VQRJj86xrfmpD5VEmG5SfiTElvsPAKjYSdXDEQxgfyTY35zpqbsk97nA==)。
    2.  鶏肉は食べやすい大きさに切る。
    3.  フライパンに鶏肉、しいたけの戻し汁(200cc)、調味料を入れ中火で熱し、煮立ったらアクを取り除く。
    4.  しょうゆを加えて混ぜ、再び落としぶたをして5〜6分煮る。
    5.  全体を混ぜ、強めの中火にして照りが出るまで煮汁をとばしながら煮る [[64]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEbH6nZ9zYJEWz7TX3_mzz5BtYobXfPVftPL1qtAj3zd0JXYxLnMlfcM2sr0ZN-ybgl76X1lgeCraqviJxoKNzrwCnZq_VQRJj86xrfmpD5VEmG5SfiTElvsPAKjYSdXDEQxgfyTY35zpqbsk97nA==)。

#### 4.2 夏野菜を使った郷土料理のレシピ（上記5つ以外）

提供された情報源には、上記5つの郷土料理以外に、夏野菜を使った郷土料理のレシピが3つ紹介されています。

*   **ずんだ餅（宮城県）**：ゆでた枝豆をすりつぶして砂糖などを混ぜて餡にした餅菓子です。餅の代わりに白玉を使った「ずんだだんご」もおすすめです。
*   **すったて（埼玉県川島町）**：味噌仕立ての冷や汁うどんです。すり鉢で胡麻と味噌を合わせ、大葉、きゅうり、茗荷などの夏野菜を加えて一緒にすり、冷たい井戸水と冷やした手打ちうどんを加えます [[66]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGz3JQxR7v-QSjunXdOtjUIVZwyg-Tv0um_0cFUc5DIt_TfwsghjjAbIELAllu4r_2a0z7peV9a9tucrSepU-c7ugeOpUvi-FIrdPT6bvYJ3Ccw1KOTb80=)。
*   **だし（山形県村山地方）**：ミョウガ、キュウリ、ナス、ショウガ、シソ、ネギ、青ナンバンなどの夏野菜をみじん切りにし、醤油とかつお節で味付けしたものです。温かいごはんにかけたり、麺や冷奴にかけたりして食べます [[67]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFlcC_ibqD90QN63sQy3OXxoWMYEuIUIBXT0Zsre85UDMjBv7kQt29GHyCHDuxZpByAIudPp_MAyvKB5lskvNXz-JAQ3TyuFx3Meu0m_WAmYrJ5ulS1vaoZaUnxfZEwigdndAjpbQYFElpbYm7FDTBgyLys8pQ=)。

これらの料理は、夏バテ対策にもなり、食欲をそそるものばかりです [[68]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEr-GlcQrjOoFgmUbh_4_-nhWzIbgKKqYmd3uQO0FfDJLZuZHZsh-Ywx8_YPfttvXjeVXWSpBAVQDDbpoB7ZYxE4f87rsLp9IFfKP4fCfXtuMysyUU49aPuAf2ONYNglVBjE-vM0shQ4yXWbf0qSsEUPPyADbApF7iVz4Cn2OCBGbY9RS6PHFGmWAxm2mFaysva)[[66]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGz3JQxR7v-QSjunXdOtjUIVZwyg-Tv0um_0cFUc5DIt_TfwsghjjAbIELAllu4r_2a0z7peV9a9tucrSepU-c7ugeOpUvi-FIrdPT6bvYJ3Ccw1KOTb80=)[[67]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFlcC_ibqD90QN63sQy3OXxoWMYEuIUIBXT0Zsre85UDMjBv7kQt29GHyCHDuxZpByAIudPp_MAyvKB5lskvNXz-JAQ3TyuFx3Meu0m_WAmYrJ5ulS1vaoZaUnxfZEwigdndAjpbQYFElpbYm7FDTBgyLys8pQ=)。

### 5. レシピの評価と改善

本調査では、具体的なレシピの考案と試作、評価、改善は行いませんでした。

### 6. レシピのレポート作成

本レポートは、調査結果をまとめたものです。

### エグゼクティブサマリー

本調査では、日本の代表的な郷土料理（石狩鍋、きりたんぽ鍋、冷や汁、もぶり飯、がめ煮）の特徴とレシピ、夏に合うスパイシーな食材・調味料の種類、郷土料理へのスパイシーなアレンジの可能性について調査しました。

調査の結果、各郷土料理には、その土地の食材や文化が反映された独自の魅力があることがわかりました。また、夏に合うスパイシーな食材や調味料は多岐にわたり、これらを活用することで、郷土料理を夏向けにアレンジし、食欲増進や疲労回復効果を期待できることが示唆されました。

特に、石狩鍋、きりたんぽ鍋、冷や汁については、情報源から具体的なスパイシーな調味料の提案（山椒、カレーパウダー、七味唐辛子、ラー油、ショウガ、ワサビなど）や、相性の良い夏野菜（カボチャ、トウモロコシ、ナス、キュウリ、オクラなど）に関する情報が得られました。

一方で、もぶり飯とがめ煮については、スパイシーなアレンジや夏野菜に関する具体的な情報は見つかりませんでした。また、郷土料理にスパイシーな食材を組み合わせる際には、風味のバランス、食材の相性、辛さの調整などに注意する必要があることが示されました。

本調査結果は、郷土料理をベースにした夏向けスパイシーレシピの開発における基礎情報として活用できると考えられます。    '''

In [48]:
client = LocalApp(article_writer, 'Article writer', 'user00')

DEBUG=False
query = '''
トピック「夏を乗り切る郷土料理のスパイシーアレンジ」
'''
_ = await client.stream(query)

トピック「夏を乗り切る郷土料理のスパイシーアレンジ」についてですね！承知いたしました。
記事を作成するためには、このトピックに関する「調査レポート」が必要です。

郷土料理のスパイシーアレンジに関する具体的なデータや事例、専門家の見解などがまとめられたレポートをご提供いただけますでしょうか？

調査レポートをお待ちしております！


In [49]:
query = f'''
{research_report}
'''
_ = await client.stream(query)

夏は暑いけれど、やっぱり美味しいものは諦めたくない！そんなあなたにピッタリのテーマ、「夏を乗り切る郷土料理のスパイシーアレンジ」について、今回は深掘りしていきましょう！🔥🌶️

---

### 🌴夏の食欲を刺激！郷土料理スパイシーアレンジの魅力✨

うだるような暑さが続く日本の夏。ついつい冷たいものばかりに手が伸びて、食欲が落ちてしまう…なんて経験、誰にでもありますよね。そんな時こそ、ピリッと刺激的な料理が恋しくなりませんか？実は日本の豊かな郷土料理には、ちょっとした工夫で夏の暑さを吹き飛ばすスパイシーな逸品に変身させるポテンシャルが秘められているんです！

### 🇯🇵奥深き郷土料理の世界とスパイスの力！🌶️

日本の郷土料理は、その土地の気候や風土、歴史、文化が色濃く反映された、まさに地域の宝物。農林水産省のウェブサイト「うちの郷土料理」には、なんと全国各地の1,365種類ものレシピが紹介されているんですよ！これだけの多様性がある郷土料理だからこそ、様々なアレンジが楽しめるんです。

そして、夏にスパイシーなものが食べたくなるのにはちゃんと理由があります。スパイスには血行を促進して疲労回復を助けたり、発汗を促して体をクールダウンさせたり、消化液の分泌を活発にして食欲を増進させる効果が期待できるんです。まさに、夏バテ対策の強い味方！伝統的な郷土料理に、ちょっと新しい刺激をプラスすることで、夏の食卓がもっと豊かになりますよ。

### 🌟注目アレンジ例：郷土料理が大変身！

調査によると、いくつかの郷土料理は特にスパイシーアレンジとの相性が抜群だとか。いくつかご紹介しますね！

*   **宮崎のソウルフード「冷や汁」が進化！**
    もともと夏にぴったりの冷や汁ですが、ワサビやミョウガ、青じそといった薬味をたっぷり加えるのはもちろん、和山椒で爽やかな辛味をプラスしたり、なんとラー油を少量加えるエスニック風アレンジも！食欲がない時でもつるっと食べられます。

*   **北海道の定番「石狩鍋」を夏らしく！**
    冬のイメージが強い石狩鍋ですが、締めにご飯とチーズ、そしてカレーパウダーを加えて「スパイシーカレーラーメン」にするアレンジがおすすめ！さらに、夏野菜のカボチャやトウモロコシなどを加えるのも良いですね。味噌とスパイスの組み合わせは意外な発見があるか

In [31]:
from vertexai import agent_engines

remote_agent = agent_engines.create(
    agent_engine=article_writer,
    display_name='article_writer',
    requirements=[
        'google-adk==0.5.0',
        'google-cloud-aiplatform[agent_engines]==1.93.0',
        'google-cloud-storage==2.19.0',
    ]
)

Deploying google.adk.agents.Agent as an application.
Identified the following requirements: {'google-cloud-aiplatform': '1.93.0', 'pydantic': '2.11.5', 'cloudpickle': '3.1.1'}
The following requirements are missing: {'pydantic', 'cloudpickle'}
The following requirements are appended: {'pydantic==2.11.5', 'cloudpickle==3.1.1'}
The final list of requirements: ['google-adk==0.5.0', 'google-cloud-aiplatform[agent_engines]==1.93.0', 'google-cloud-storage==2.19.0', 'pydantic==2.11.5', 'cloudpickle==3.1.1']
Using bucket etsuji-15pro-poc
Wrote to gs://etsuji-15pro-poc/agent_engine/agent_engine.pkl
Writing to gs://etsuji-15pro-poc/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://etsuji-15pro-poc/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/879055303739/locations/us-central1/reasoningEngines/6955994342427197440/operations/4272299042727264256
View progress and logs at https://console.cloud.google.com/logs/

In [32]:
AGENTSPACE_APP_ID='agentspace-agentengine-int_1748464391060' # Your Agentangine App ID

payload = f'''{{
    "displayName": "読み物記事・作成エージェント",
    "description": "調査レポートを元に気軽な読み物記事を作成します。",
    "icon": {{
      "uri": ""
    }},
    "adk_agent_definition": {{
      "tool_settings": {{
        "tool_description": "調査レポートを元に気軽な読み物記事を作成します。"
      }},
      "provisioned_reasoning_engine": {{
        "reasoning_engine": "{remote_agent.resource_name}"
      }},
    }}
  }}
'''

!curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  -H "X-Goog-User-Project: {PROJECT_ID}" \
    "https://discoveryengine.googleapis.com/v1alpha/projects/{PROJECT_ID}/locations/global/collections/default_collection/engines/{AGENTSPACE_APP_ID}/assistants/default_assistant/agents" \
  -d '{payload}'

{
  "name": "projects/879055303739/locations/global/collections/default_collection/engines/agentspace-agentengine-int_1748464391060/assistants/default_assistant/agents/12589930513912923434",
  "displayName": "読み物記事・作成エージェント",
  "description": "調査レポートを元に気軽な読み物記事を作成します。",
  "icon": {
    "uri": ""
  },
  "adkAgentDefinition": {
    "toolSettings": {
      "toolDescription": "調査レポートを元に気軽な読み物記事を作成します。"
    },
    "provisionedReasoningEngine": {
      "reasoningEngine": "projects/879055303739/locations/us-central1/reasoningEngines/6955994342427197440"
    }
  },
  "state": "CONFIGURED"
}


In [51]:
agent_engines.update(
    agent_engine=article_writer,
    resource_name=remote_agent.resource_name
)

Deploying google.adk.agents.Agent as an application.
Using bucket etsuji-15pro-poc
Wrote to gs://etsuji-15pro-poc/agent_engine/agent_engine.pkl
Update Agent Engine backing LRO: projects/879055303739/locations/us-central1/reasoningEngines/6955994342427197440/operations/4927291313532960768
Agent Engine updated. Resource name: projects/879055303739/locations/us-central1/reasoningEngines/6955994342427197440


resource name: projects/879055303739/locations/us-central1/reasoningEngines/6955994342427197440